### Testes Cuthill-McKee

In [13]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import csgraph_from_dense
from scipy.sparse.csgraph._reordering import reverse_cuthill_mckee as rcm
from itertools import permutations

def generate_symmetric_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)
    return csr_matrix((data, (row, col)), shape=(size, size))

def brute_force_cuthill_mckee(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_bandwidth = float('inf')

    # Gera todas as permutações possíveis dos índices
    for permutation in permutations(range(size)):
        # Aplica a permutação à matriz
        permuted_matrix = matrix[permutation, :][:, permutation]

        # Converte para uma matriz densa (necessária para reverse_cuthill_mckee)
        dense_matrix = permuted_matrix.toarray()

        # Aplica Cuthill-McKee reverso
        rcm_permutation = rcm(csgraph_from_dense(dense_matrix))[0]

        # Calcula a largura de banda
        current_bandwidth = np.max(np.abs(np.diff(np.where(dense_matrix))))

        # Atualiza a melhor permutação se encontrarmos uma com menor largura de banda
        if current_bandwidth < best_bandwidth:
            best_bandwidth = current_bandwidth
            best_permutation = permutation

    return best_permutation, best_bandwidth

# Configurações
size = 5
matrix = generate_symmetric_sparse_matrix(size)

# Força bruta Cuthill-McKee
best_permutation, best_bandwidth = brute_force_cuthill_mckee(matrix)

print("Matriz Original:")
print(matrix.toarray())
print("\nMelhor Permutação (Cuthill-McKee):", best_permutation)
print("Largura de Banda Resultante:", best_bandwidth)


Matriz Original:
[[5 0 9 6 0]
 [9 0 0 0 0]
 [0 7 0 0 0]
 [0 0 3 0 7]
 [0 0 0 1 9]]

Melhor Permutação (Cuthill-McKee): (0, 2, 4, 3, 1)
Largura de Banda Resultante: 2


In [22]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import csgraph_from_dense, reverse_cuthill_mckee
from itertools import permutations
import time

def generate_symmetric_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)
    return csr_matrix((data, (row, col)), shape=(size, size))

def brute_force_cuthill_mckee(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_bandwidth = float('inf')

    # Gera todas as permutações possíveis dos índices
    for permutation in permutations(range(size)):
        # Aplica a permutação à matriz
        permuted_matrix = matrix[permutation, :][:, permutation]

        # Converte para uma matriz densa (necessária para reverse_cuthill_mckee)
        dense_matrix = permuted_matrix.toarray()

        # Mede o tempo de execução para Cuthill-McKee reverso
        start_time = time.time()
        rcm_permutation = reverse_cuthill_mckee(csgraph_from_dense(dense_matrix))[0]
        elapsed_time = time.time() - start_time

        # Calcula a largura de banda
        current_bandwidth = np.max(np.abs(np.diff(np.where(dense_matrix))))

        # Atualiza a melhor permutação se encontrarmos uma com menor largura de banda
        if current_bandwidth < best_bandwidth:
            best_bandwidth = current_bandwidth
            best_permutation = permutation

    return best_permutation, best_bandwidth, elapsed_time

# Configurações
for size in range(5, 7):
    matrix = generate_symmetric_sparse_matrix(size)

    # Cuthill-McKee
    start_time_cmk = time.time()
    cmk_permutation = reverse_cuthill_mckee(csgraph_from_dense(matrix.toarray()))[0]
    elapsed_time_cmk = time.time() - start_time_cmk

    # Cuthill-McKee reverso
    start_time_rcm = time.time()
    rcm_permutation, _, elapsed_time_rcm = brute_force_cuthill_mckee(matrix)
    elapsed_time_rcm = time.time() - start_time_rcm

    # Força bruta Cuthill-McKee
    start_time_bf = time.time()
    bf_permutation, bf_bandwidth, elapsed_time_bf = brute_force_cuthill_mckee(matrix)

    print(f"\nTamanho da Matriz: {size}")
    print("Matriz Original:")
    print(matrix.toarray())
    print("Cuthill-McKee:", cmk_permutation, f"Tempo Gasto: {elapsed_time_cmk:.6f} segundos")
    print("Cuthill-McKee Reverso (Bruta):", rcm_permutation, f"Tempo Gasto: {elapsed_time_rcm:.6f} segundos")
    print("Força Bruta Cuthill-McKee:", bf_permutation, f"Tempo Gasto: {elapsed_time_bf:.6f} segundos")
    print("Largura de Banda Resultante (Bruta):", bf_bandwidth)



Tamanho da Matriz: 5
Matriz Original:
[[ 2  0  0  0  0]
 [ 0  0  0 11  0]
 [ 3  0  0  1  6]
 [ 0  6  6  0  4]
 [ 0  0  0  0  0]]
Cuthill-McKee: 0 Tempo Gasto: 0.000996 segundos
Cuthill-McKee Reverso (Bruta): (0, 2, 3, 4, 1) Tempo Gasto: 0.080359 segundos
Força Bruta Cuthill-McKee: (0, 2, 3, 4, 1) Tempo Gasto: 0.000000 segundos
Largura de Banda Resultante (Bruta): 2

Tamanho da Matriz: 6
Matriz Original:
[[ 2  0  0  0  0  1]
 [ 7  0  0  0  0  0]
 [ 0  9  0  0  0  0]
 [ 0  9  1 12  0  0]
 [ 0  0  0  0  8  0]
 [10  0  0  0  0  0]]
Cuthill-McKee: 4 Tempo Gasto: 0.000000 segundos
Cuthill-McKee Reverso (Bruta): (4, 3, 2, 1, 0, 5) Tempo Gasto: 0.471362 segundos
Força Bruta Cuthill-McKee: (4, 3, 2, 1, 0, 5) Tempo Gasto: 0.000000 segundos
Largura de Banda Resultante (Bruta): 1


### Um proposição heurística

Este código normaliza os números da matriz original entre 0 e 1, gera todas as permutações possíveis dos índices e escolhe a permutação que minimiza os somatórios das linhas. A matriz resultante é então rearranjada de acordo com a melhor permutação.

In [24]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import pandas as pd

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def normalize_and_reduce(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_row_sums = None

    # Normaliza os números da matriz entre 0 e 1
    normalized_matrix = matrix / np.max(matrix)

    # Se a matriz for unidimensional, não é necessário fazer permutações
    if normalized_matrix.ndim == 1:
        best_permutation = np.arange(size)
    else:
        # Gera todas as permutações possíveis dos índices
        for permutation in np.argsort(normalized_matrix.toarray().T, axis=1):
            # Aplica a permutação à matriz
            permuted_matrix = normalized_matrix[:, permutation]

            # Calcula os somatórios das linhas
            row_sums = np.sum(permuted_matrix, axis=1)

            # Atualiza a melhor permutação se encontrarmos uma com somatórios menores que 0.5
            if best_permutation is None or np.sum(row_sums < 0.5) < np.sum(best_row_sums < 0.5):
                best_permutation = permutation
                best_row_sums = row_sums

    # Aplica a melhor permutação à matriz original
    reduced_matrix = matrix[:, best_permutation]

    return best_permutation, reduced_matrix

# Lista para armazenar os DataFrames de cada iteração
dfs = []

# Loop de tamanhos de matriz de 5 a 50, de 5 em 5
for size in range(1000, 20001, 1000):
    sparse_matrix = generate_sparse_matrix(size)

    # Mede o tempo de execução para a abordagem de força bruta
    start_time = time.time()
    best_permutation, reduced_matrix = normalize_and_reduce(sparse_matrix)
    elapsed_time = time.time() - start_time

    # Cria um DataFrame com os resultados
    df = pd.DataFrame({'Tamanho da Matriz': [size], 'Tempo Gasto (Heurística)': [elapsed_time]})
    dfs.append(df)

# Concatena todos os DataFrames em um único DataFrame
results_df = pd.concat(dfs, ignore_index=True)

# Imprime o DataFrame
print(results_df)


    Tamanho da Matriz  Tempo Gasto (Heurística)
0                1000                  0.300342
1                2000                  0.732874
2                3000                  1.448493
3                4000                  2.183799
4                5000                  3.359456
5                6000                  4.964066
6                7000                  6.210109
7                8000                  8.144787
8                9000                 10.822167
9               10000                 13.480871
10              11000                 16.111915
11              12000                 19.365576
12              13000                 22.352625
13              14000                 25.204412
14              15000                 30.092970
15              16000                 37.967810
16              17000                 41.210262
17              18000                 48.791801
18              19000                 57.659708
19              20000                 73

### Cuthill-McKee e Cuthill-McKee Reverso 

In [4]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import pandas as pd
from scipy.sparse.csgraph import reverse_cuthill_mckee

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def normalize_and_reduce(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_row_sums = None

    # Normaliza os números da matriz entre 0 e 1
    #normalized_matrix = matrix / np.max(matrix)

    # Se a matriz for unidimensional, não é necessário fazer permutações
    #if normalized_matrix.ndim == 1:
     #   best_permutation = np.arange(size)
    #else:
        # Gera todas as permutações possíveis dos índices
        #for permutation in np.argsort(normalized_matrix.toarray().T, axis=1):
            # Aplica a permutação à matriz
            #permuted_matrix = normalized_matrix[:, permutation]

            # Calcula os somatórios das linhas
            #row_sums = np.sum(permuted_matrix, axis=1)

            # Atualiza a melhor permutação se encontrarmos uma com somatórios menores que 0.5
            #if best_permutation is None or np.sum(row_sums < 0.5) < np.sum(best_row_sums < 0.5):
               # best_permutation = permutation
               # best_row_sums = row_sums

    # Aplica a melhor permutação à matriz original
  #  reduced_matrix = matrix[:, best_permutation]

  #  return best_permutation, reduced_matrix

def bandwidth_reduction(matrix):
    sparse_matrix = csr_matrix(matrix)
    permutation = reverse_cuthill_mckee(sparse_matrix, symmetric_mode=True)
    reduced_matrix = matrix[:, permutation]
    return reduced_matrix

# Lista para armazenar os DataFrames de cada iteração
dfs = []

# Loop de tamanhos de matriz de 4.000 a 40.000, de 4.000 em 4.000
for size in range(4000, 40001, 4000):
    sparse_matrix = generate_sparse_matrix(size)

    # Mede o tempo de execução para a abordagem de força bruta
   # start_time_force_bruta = time.time()
  #  _, _ = normalize_and_reduce(sparse_matrix)
  #  elapsed_time_force_bruta = time.time() - start_time_force_bruta

    # Mede o tempo de execução para Cuthill-McKee
    start_time_cmk = time.time()
    _ = bandwidth_reduction(sparse_matrix)
    elapsed_time_cmk = time.time() - start_time_cmk

    # Mede o tempo de execução para Cuthill-McKee Reverso
    start_time_rcm = time.time()
    _ = bandwidth_reduction(sparse_matrix.toarray())
    elapsed_time_rcm = time.time() - start_time_rcm

    # Cria um DataFrame com os resultados
    df = pd.DataFrame({
        'Tam. Matriz': [size],
       # 'Tempo(Força Bruta)': [elapsed_time_force_bruta],
        'Tempo(Cuthill-McKee)': [elapsed_time_cmk],
            'Tempo(Cuthill-McKee Reverso)': [elapsed_time_rcm]
    })
    dfs.append(df)

# Concatena todos os DataFrames em um único DataFrame
results_df = pd.concat(dfs, ignore_index=True)

# Imprime os resultados em uma única linha
print(results_df.to_string(index=False))


 Tam. Matriz  Tempo(Cuthill-McKee)  Tempo(Cuthill-McKee Reverso)
        4000              0.109960                      0.429072
        8000              0.000000                      1.792260
       12000              0.015619                      3.896586
       16000              0.021400                      8.782393
       20000              0.031273                     15.551048
       24000              0.062690                     28.816832
       28000              0.130504                     49.446158
       32000              0.122569                    106.258461
       36000              0.110106                   1292.167451
       40000              0.135207                   2263.546423


### Guloso

In [28]:
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd
import time

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def bandwidth(matrix):
    # Calcula a largura de banda da matriz
    row_indices, col_indices = matrix.nonzero()
    bandwidth = np.max(np.abs(row_indices - col_indices))
    return bandwidth

def greedy_bandwidth_reduction(matrix):
    size = matrix.shape[0]
    permutation = np.arange(size)
    original_bandwidth = bandwidth(matrix)

    for i in range(size):
        best_bandwidth_reduction = 0
        best_position = i

        for j in range(i, size):
            # Tenta mover a linha/coluna j para a posição i e calcula a redução na largura de banda
            matrix[:, [i, j]] = matrix[:, [j, i]]
            matrix[[i, j], :] = matrix[[j, i], :]
            new_bandwidth = bandwidth(matrix)

            # Se a mudança reduzir a largura de banda, atualiza as variáveis
            if original_bandwidth - new_bandwidth > best_bandwidth_reduction:
                best_bandwidth_reduction = original_bandwidth - new_bandwidth
                best_position = j

            # Reverte a mudança para testar a próxima posição
            matrix[:, [i, j]] = matrix[:, [j, i]]
            matrix[[i, j], :] = matrix[[j, i], :]

        # Move a linha/coluna selecionada para a posição correta
        matrix[:, [i, best_position]] = matrix[:, [best_position, i]]
        matrix[[i, best_position], :] = matrix[[best_position, i], :]
        permutation[i], permutation[best_position] = permutation[best_position], permutation[i]

    return matrix, permutation

# Lista para armazenar os DataFrames de cada iteração
dfs_greedy = []

# Loop de tamanhos de matriz de 100 a 200, de 10 em 10
for size in range(100, 201, 10):
    sparse_matrix = generate_sparse_matrix(size)

    # Mede o tempo de execução para o algoritmo guloso
    start_time_greedy = time.time()
    _, _ = greedy_bandwidth_reduction(sparse_matrix.copy())
    elapsed_time_greedy = time.time() - start_time_greedy

    # Cria um DataFrame com os resultados
    df_greedy = pd.DataFrame({
        'Tamanho da Matriz': [size],
        'Tempo Gasto (Guloso)': [elapsed_time_greedy],
    })
    dfs_greedy.append(df_greedy)

# Concatena todos os DataFrames em um único DataFrame
results_df_greedy = pd.concat(dfs_greedy, ignore_index=True)

# Imprime os resultados em uma única linha
print(results_df_greedy.to_string(index=False))


 Tamanho da Matriz  Tempo Gasto (Guloso)
               100              7.934230
               110              9.951677
               120             12.454042
               130             14.932182
               140             18.234554
               150             21.203526
               160             25.883170
               170             28.967700
               180             33.814210
               190             38.528729
               200             45.742847


In [4]:
import numpy as np
import time
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def bandwidth_reduction(matrix):
    sparse_matrix = csr_matrix(matrix)
    permutation = reverse_cuthill_mckee(sparse_matrix, symmetric_mode=True)
    reduced_matrix = matrix[:, permutation]
    return reduced_matrix

# Lista para armazenar os DataFrames de cada iteração
dfs = []

# Tempo acumulado total
total_elapsed_time = 0

# Loop de tamanhos de matriz de 10.000.000 a 100.000.000, de 10.000.000 em 10.000.000.
for size in range(10000000, 100000001, 10000000):
    sparse_matrix = generate_sparse_matrix(size)

    # Mede o tempo de execução para Cuthill-McKee
    start_time_cmk = time.time()
    _ = bandwidth_reduction(sparse_matrix)
    elapsed_time_cmk = time.time() - start_time_cmk

    # Acumula o tempo total
    total_elapsed_time += elapsed_time_cmk

    # Cria um DataFrame com os resultados
    df = pd.DataFrame({
        'Tam. Matriz': [size],
        'Tempo(Cuthill-McKee)': [elapsed_time_cmk],
        'Tempo Acumulado': [total_elapsed_time]
    })
    dfs.append(df)

# Concatena todos os DataFrames em um único DataFrame
results_df = pd.concat(dfs, ignore_index=True)

# Imprime os resultados em uma única linha
print(results_df.to_string(index=False))


 Tam. Matriz  Tempo(Cuthill-McKee)  Tempo Acumulado
    10000000              7.241643         7.241643
    20000000             18.262763        25.504406
    30000000             33.788611        59.293017
    40000000             42.515649       101.808666
    50000000             53.535936       155.344602
    60000000             75.227901       230.572503
    70000000             94.274439       324.846941
    80000000            137.449201       462.296142
    90000000            146.307438       608.603580
   100000000            258.368767       866.972347


### Força Bruta

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import pandas as pd
from scipy.sparse.csgraph import reverse_cuthill_mckee

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def normalize_and_reduce(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_row_sums = None

def bandwidth_reduction(matrix):
    sparse_matrix = csr_matrix(matrix)
    permutation = reverse_cuthill_mckee(sparse_matrix, symmetric_mode=True)
    reduced_matrix = matrix[:, permutation]
    return reduced_matrix

# Lista para armazenar os DataFrames de cada iteração
dfs = []

# Loop de tamanhos de matriz de 1.000 a 20.000, de 1.000 em 1.000
for size in range(1000, 20001, 1000):
    sparse_matrix = generate_sparse_matrix(size)

    Mede o tempo de execução para a abordagem de força bruta
start_time_force_bruta = time.time()
  _, _ = normalize_and_reduce(sparse_matrix)
elapsed_time_force_bruta = time.time() - start_time_force_bruta

       # Cria um DataFrame com os resultados
    df = pd.DataFrame({
        'Tam. Matriz': [size],
       'Tempo(Força Bruta)': [elapsed_time_force_bruta],
        
    dfs.append(df)

# Concatena todos os DataFrames em um único DataFrame
results_df = pd.concat(dfs, ignore_index=True)

# Imprime os resultados em uma única linha
print(results_df.to_string(index=False))


### Cuthill-McKee e Cuthill-McKee Reverso 

In [3]:
import numpy as np
from scipy.sparse import csr_matrix
import time
import pandas as pd
from scipy.sparse.csgraph import reverse_cuthill_mckee

def generate_sparse_matrix(size):
    row = np.random.randint(0, size, size * 2)
    col = np.random.randint(0, size, size * 2)
    data = np.random.randint(1, 10, size * 2)  # Números inteiros entre 1 e 10
    return csr_matrix((data, (row, col)), shape=(size, size))

def normalize_and_reduce(matrix):
    size = matrix.shape[0]
    best_permutation = None
    best_row_sums = None

def bandwidth_reduction(matrix):
    sparse_matrix = csr_matrix(matrix)
    permutation = reverse_cuthill_mckee(sparse_matrix, symmetric_mode=True)
    reduced_matrix = matrix[:, permutation]
    return reduced_matrix

# Lista para armazenar os DataFrames de cada iteração
dfs = []

# Loop de tamanhos de matriz de 4.000 a 40.000, de 4.000 em 4.000
for size in range(4000, 40001, 4000):
    sparse_matrix = generate_sparse_matrix(size)

    # Mede o tempo de execução para Cuthill-McKee
    start_time_cmk = time.time()
    _ = bandwidth_reduction(sparse_matrix)
    elapsed_time_cmk = time.time() - start_time_cmk

    # Mede o tempo de execução para Cuthill-McKee Reverso
    start_time_rcm = time.time()
    _ = bandwidth_reduction(sparse_matrix.toarray())
    elapsed_time_rcm = time.time() - start_time_rcm

    # Cria um DataFrame com os resultados
    df = pd.DataFrame({
        'Tam. Matriz': [size],
       # 'Tempo(Força Bruta)': [elapsed_time_force_bruta],
        'Tempo(Cuthill-McKee)': [elapsed_time_cmk],
            'Tempo(Cuthill-McKee Reverso)': [elapsed_time_rcm]
    })
    dfs.append(df)

# Concatena todos os DataFrames em um único DataFrame
results_df = pd.concat(dfs, ignore_index=True)

# Imprime os resultados em uma única linha
print(results_df.to_string(index=False))


 Tam. Matriz  Tempo(Cuthill-McKee)  Tempo(Cuthill-McKee Reverso)
         100              0.017997                      0.005711
         200              0.000000                      0.000996
         300              0.000000                      0.002264
         400              0.001008                      0.001005
         500              0.000000                      0.002002
         600              0.001004                      0.002498
         700              0.000000                      0.005532
         800              0.000000                      0.006572
         900              0.001231                      0.009028
        1000              0.000000                      0.011093
